#**Maestría en Analítica de Negocios**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

####**Filtrado de datos (data leakage) / Pipeline / Curvas de Aprendizaje**

In [1]:
# Librerías básicas que estaremos requiriendo en la mayoría de las actividades. 
# Recuerda usar el # para documentar tu código dentro de estas celdas de Código.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns   # para un mejor despliegue de los gráficos

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
# Queremos accesar el archivo que está en la carpeta "sample_data" en la cual nos encontramos de manera  
# predeterminada y que podemos verificar con el siguiente comando que nos permite listar sus archivos
# y directorios:

!ls

In [ ]:
# La siguiente instrucción nos permite adentrarnos en dicha carpeta y de nuevo listamos lo que hay dentro de ella:

%cd sample_data/

!ls

In [ ]:
# En particular, los datos para el entrenamiento los encontramos obviamente en el siguiente archivo, el cual procedemos a cargar: 

data = pd.read_csv("california_housing_train.csv", sep=",")
data.head()

De cualquier de las ligas dadas anteriormente, sabemos que la variable de salida es el precio medio (mediana) de la casa en dólares estadounidenses, "median_house_value".

In [ ]:
data.info()

In [6]:
sns.set(rc={'figure.figsize':(12,10)})   # (ancho-columnas, altura-renglones) Ajustemos el tamaño de la ventana 
                                         # que desplegará los gráficos usando la librería de seaborn (sns).

In [ ]:
fig, axes = plt.subplots(3, 3)    # definimos una ventana de 3x3 nichos para incluir en cada uno de ellos un gráfico.
for k in range(0,9):
  plt.subplot(3,3,k+1)     # los nichos para cada histograma se numeran iniciando en 1 y no en 0.
  plt.hist(data[data.columns[k]], bins=20)     # datatrain.columns nos devuelve una lista con los nombres de las columnas.
  plt.xlabel(data.columns[k])
plt.show()

In [ ]:
data.describe()


Puedes nuevamente observar de esta tabla varias de las características que mencionamos previamente a partir de los histogramas.

In [9]:
X1 =data[['total_bedrooms']]
yy = data[['median_house_value']]

In [10]:
sns.set(rc={'figure.figsize':(6,4)})

In [ ]:
np.sqrt(X1).hist();

#**Modelo-1**

In [ ]:
m = LinearRegression()

fit1 = m.fit(X1, yy)

preds1 = fit1.predict(X1)
preds1 = np.sqrt(mean_squared_error(yy, preds1))

print("RMSE: %.2f" % (preds1))

###**¿Cuál es el error de diseño de este modelo, en caso de que exista, de acuerdo a la metodología de aprendizaje automático (machine learning)?**

#**Modelo-2**

In [ ]:
m = LinearRegression()

X2 = np.sqrt(X1)

fit2 = m.fit(X2, yy)

preds2 = fit2.predict(X2)
preds2 = np.sqrt(mean_squared_error(yy, preds2))

print("RMSE: %.2f" % (preds2))

###**¿Cuál es el error de diseño de este modelo, en caso de que exista, de acuerdo a la metodología de aprendizaje automático (machine learning)?**

#**Modelo-3**

In [14]:
semilla = 11
val_size = 0.2

In [ ]:
X2 = np.sqrt(X1)

X_train, X_val, y_train, y_val = train_test_split(X2, yy, test_size=val_size, random_state=semilla)



m = LinearRegression()

fit3 = m.fit(X_train, y_train)

preds3 = fit3.predict(X_val)
preds3 = np.sqrt(mean_squared_error(y_val, preds3))

print("RMSE: %.2f" % (preds3))

###**¿Cuál es el error de diseño de este modelo, en caso de que exista, de acuerdo a la metodología de aprendizaje automático (machine learning)?**

###NOTA: Observa que al particionar el conjunto original de train en 80% y 20%, ya tenemos ahora el conjunto de entrenamiento y el de validación. El archivo restante que se encuentra en la carpeta del Google-Colab, llamado "california_housing_test.csv", será el conjunto de prueba, completando lo tres conjuntos con los cuales estaremos trabajando.

#**Modelo-4**

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X1, yy, test_size=val_size, random_state=semilla)

X4_train = np.sqrt(X_train)
X4_val = np.sqrt(X_val)



m = LinearRegression()

fit4 = m.fit(X4_train, y_train)

preds4 = fit4.predict(X4_val)
preds4 = np.sqrt(mean_squared_error(y_val, preds4))

print("RMSE: %.2f" % (preds4))

RMSE: 114709.53


###**¿Cuál es el error de diseño de este modelo, en caso de que exista, de acuerdo a la metodología de aprendizaje automático (machine learning)?**

#**Modelo-5**

In [17]:
# Supongamos que también deseamos escalar los datos mediante la transformación (x-miu)/std

from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, yy, test_size=val_size, random_state=semilla)

X5_train = np.sqrt(X_train)
X5_train = (X5_train - np.mean(X5_train)) / np.std(X5_train)

X5_val = np.sqrt(X_val)
X5_val = (X5_val - np.mean(X5_val)) / np.std(X5_val)



m = LinearRegression()

fit5 = m.fit(X5_train, y_train)

preds5 = fit5.predict(X5_val)
preds5 = np.sqrt(mean_squared_error(y_val, preds5))

print("RMSE: %.2f" % (preds5))

###**¿Cuál es el error de diseño de este modelo, en caso de que exista, de acuerdo a la metodología de aprendizaje automático (machine learning)?**

#**Modelo-6: Usando Pipelines para evitar el filtrado de información (data leakage)**




###Veamos a continuación la manera correcta de llevar a cabo las transformaciones mediante el uso de la clase Pipeline de scikit-learn. La clase Pipeline nos permitirá encapsular una serie de transformaciones para llevar a cabo el entrenamiento de un modelo, apoyando a evitar el filtrado de información.

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer   
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, yy, test_size=val_size, random_state=semilla)

# Transformaciones a factores numéricos de entrada:
num_pipeline = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median')),
                                 ('sqrt', FunctionTransformer(np.sqrt)),                                 
                                 ('escalaNum', StandardScaler())])   

num_pipeline_nombres = ['total_bedrooms']


# Aplicamos dichas transformaciones a las variables indicadas y el resto las dejamos igual:
columnasTransformer = ColumnTransformer(transformers = [('vars_num_pipeline', num_pipeline, num_pipeline_nombres)
                                                        ],
                                        remainder='passthrough')


m = LinearRegression()

# Aplicamos las transformaciones al conjunto de entrenamiento:
XtrainFit = columnasTransformer.fit(X_train)   # Generamos la información necesaria con el conjunto de entrenamiento
                                               # para evitar el filtrado de información.

XtrainTransf = XtrainFit.transform(X_train)    # y ahora aplicamos dichas transformaciones al conjunto de entrenamiento,
XvalTransf  =  XtrainFit.transform(X_val)      # y también al conjunto de validación (con la información de los datos de Train).


fit6 = m.fit(XtrainTransf, y_train)

preds6 = fit6.predict(XvalTransf)
preds6 = np.sqrt(mean_squared_error(y_val, preds6))

print("RMSE: %.2f" % (preds6))

#**Curvas de Aprendizaje**

In [21]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import RepeatedKFold

In [ ]:
modeloLC = LinearRegression()

XtvTransf = XtrainFit.transform(X1)   # Como usaremos Cross-Validation, usamos el conjunto original de Train, X1 y sus valores reales yy.

delta_train_sz = np.linspace(0.1, 1.0, 20)

cvLC = RepeatedKFold(n_splits=10, n_repeats=5, random_state=semilla)

train_sizes, train_scores, valid_scores = learning_curve(estimator=modeloLC, 
                                                        X=XtvTransf, 
                                                        y=yy,
                                                        cv=cvLC, 
                                                        train_sizes=delta_train_sz,
                                                        scoring='neg_root_mean_squared_error')



# Obtengamos la gráfica de las curvas de aprendizaje 
# cuando se incrementa el tamaño de la muestra:

train_mean = -np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
valid_mean = -np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(train_sizes, valid_mean, color='red', marker='o', markersize=5, label='Validation')
plt.fill_between(train_sizes, valid_mean + valid_std, valid_mean - valid_std, alpha=0.15, color='red')

plt.title('Función learning_curve()')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('RMSE')
plt.grid()
plt.legend(loc='lower right')
plt.show()

###NOTA: En el caso de scikit-learn, recuerda que calcula el valor negativo de RMSE.

### Consulta la documentación correspondiente:

https://scikit-learn.org/stable/modules/model_evaluation.html

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html


#**Modelo final con el conjunto de Prueba (Test)**

In [ ]:
datatest = pd.read_csv("california_housing_test.csv", sep=",")   # cargamos los datos de la carpeta del Google-Colab, observa que
                                                                 # hasta este momento no se habían usado estos datos.
datatest.shape

In [24]:
Xtest =datatest[['total_bedrooms']]
ytest = datatest[['median_house_value']]

In [25]:
mf = LinearRegression()

XtvTransf = XtrainFit.transform(X1)   # Observa que al obtener el entrenamiento final, usamos los datos tanto de Train como de Validation X1,
                                      # esto permitirá todavía usar una mayor cantidad de información para aprender mejor. Sin embargo,
                                      # seguimos usando la información solo del conjunto Train para evitar el filtrado de información.

XtestTransf  =  XtrainFit.transform(Xtest)  

fitf = mf.fit(XtvTransf, yy)

predsf = fitf.predict(XtestTransf)
predsf = np.sqrt(mean_squared_error(ytest, predsf))

print("RMSE: %.2f" % (predsf))

RMSE: 112733.28


In [ ]:
print("\tModelo1\t\tModelo2\t\tModelo3\t\tModelo4\t\tModelo5\t\tModelo6\t\tModeloF")

print("RMSE:  %.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t%.2f\t" % (preds1,preds2,preds3,preds4,preds5,preds6,predsf) )



###Conclusiones ...